# 1.0 读取CSV

In [ ]:
# Create data frame of next 500 rows with labeled columns
vt_data_next500 = pd.read_csv("vt_tax_data_2016.csv", 
                       		  nrows = 500,
                       		  skiprows = 500,
                       		  header = None,
								names = list(vt_data_first500.columns)
                       		  )


##### Set custom NA values

In [ ]:
# Create dict specifying that 0s in zipcode are NA values
null_values = {'zipcode': 0}

# Load csv using na_values keyword argument
data = pd.read_csv("vt_tax_data_2016.csv", 
                   na_values = null_values)


In [ ]:
tax_data = pd.read_csv("us_tax_data_2016.csv",
                       na_values={"zipcode" : 0})
print(tax_data[tax_data.zipcode.isna()])


##### error_bad_lines=False, 跳过不能读取 parse的行
##### warn_bad_lines=True，  查看跳过了多少行，显示bad行的信息

In [ ]:
tax_data = pd.read_csv("us_tax_data_2016_corrupt.csv",
                       error_bad_lines=False,
                       warn_bad_lines=True)

# 2.1 读取表格excel sheet


In [ ]:
 # Get the second sheet by position index
survey_data_sheet2 = pd.read_excel('fcc_survey.xlsx',
                                   sheet_name=1)
# Get the second sheet by name
survey_data_2017 = pd.read_excel('fcc_survey.xlsx',
                                 sheet_name='2017')
print(survey_data_sheet2.equals(survey_data_2017))


# 2.2 bool值读取及设置

In [ ]:
# Load file with Yes as a True value and No as a False value
survey_subset = pd.read_excel("fcc_survey_yn_data.xlsx",
                              dtype={"HasDebt": bool,
                              "AttendedBootCampYesNo": bool},
                              true_values = ['Yes'],
                              false_values = ['No'])

# View the data
print(survey_subset.head())

# 2.3 转化成时间格式 pars_dates = [] or {字典}

## pd.to_datetime() to convert strings(non-standard datetime) to dates  .

In [ ]:
# Create dict of columns to combine into new datetime column
datetime_cols = {"Part2Start": ['Part2StartDate', 'Part2StartTime']}


# Load file, supplying the dict to parse_dates
survey_data = pd.read_excel("fcc_survey_dts.xlsx",
                            parse_dates = datetime_cols)


survey_data["Part2EndTime"] = pd.to_datetime(survey_data["Part2EndTime"], 
                                             format="%m%d%Y %H:%M:%S")

# 3.0 链接数据库

#### 用sqlalchemy包, 链接并创建SQL引擎 create_engine

In [ ]:
from sqlalchemy import create_engine


# Create the database engine
engine = create_engine("sqlite:///data.db")

# Create a SQL query to load the entire weather table
query = """
SELECT * 
  FROM weather;
"""

# Load weather with the SQL query
weather = pd.read_sql(query, engine)

# View the first few rows of data
print(weather.head())

# 4.0 JSON文件的接入

## 4.1 pd.read_json( '文件名.json' , orient = 'split')

第一参数就是json文件路径或者json格式的字符串。

第二参数orient是表明预期的json字符串格式, orient = 包括“split", 'index', 'records', 'columns', 'values' 详细查看 http://www.manongjc.com/article/50826.html

In [ ]:
try:
    # Load the JSON with orient specified
    df = pd.read_json("dhs_report_reformatted.json",
                      orient = 'split')
    
    # Plot total population in shelters over time
    df["date_of_census"] = pd.to_datetime(df["date_of_census"])
    df.plot(x="date_of_census", 
            y="total_individuals_in_shelter")
    plt.show()
    
except ValueError:
    print("pandas could not parse the JSON.")

## 4.2 APIs 
接入其它程序，在不知道数据库details时获取数据

#### requests.get( url_string, params, headers)
params 要到API-document里查看
headers 如果有保密措施”authentication“，需要取得授权
#### data = response.json() 返回.json 文档，字典格式
#### pd.DataFrame(data) 用dataframe打开json文档

In [3]:
import requests
import pandas as pd 

api_url = "https://api.yelp.com/v3/businesses/search"

# Create dictionary to query API for cafes in NYC
parameters = {'term' :'cafe',
          	  'location': 'NYC'}

# Create dictionary that passes Authorization and key string, headers 和 params都是字典格式的参数
headers = {'Authorization': "Bearer {}".format(api_key)}

# Get data about NYC cafes from the Yelp API
response = requests.get(api_url, 
                headers=headers, 
                params=params)

# Extract JSON data from the response， 返回的是字典格式，所以不能用pd.read_json(), read_json()只能读取string格式。
data = response.json()

# Load data to a data frame, 选取‘businesses'这个key的所有values
cafes = pd.DataFrame(data['businesses'])
 
# View the data's dtypes
print(cafes.head())

NameError: name 'api_key' is not defined

 ## 4.3 处理Nested json
 A feature of JSON data is that it can be nested: an attribute's value can consist of attribute-value pairs. （详见本课chapter4 .json图）

### Flatten nested JSONs
##### json_normalize()   
##### pandas.json_normalize(data:Union[Dict, List[Dict]], record_path:Union[str, List, NoneType] = None, meta:Union[str, List[Union[str, List[str]]], NoneType] = None, meta_prefix:Union[str, NoneType] = None, record_prefix:Union[str, NoneType] = None, errors:Union[str, NoneType] = 'raise', sep:str = '.', max_level:Union[int, NoneType] = None) → 'DataFrame'

data：dict 或 list of dicts
未序列化的JSON對象。

record_path：str 或 list of str, 默認為 None
每個對象到記錄列表的路徑。如果未通過，則數據將假定為記錄數組。

meta：list of paths (str 或 list of str), 默認為 None
用作結果表中每個記錄的元數據的字段。

meta_prefix：str, 默認為 None
如果為True，則前綴記錄帶有點(？)路徑，例如foo.bar.field，如果meta為[‘foo’，‘bar’]。

record_prefix：str, 默認為 None
如果為True，則前綴記錄帶有點(？)路徑，例如foo.bar.field，如果記錄的路徑是[‘foo’，‘bar’]。

errors：{‘raise’, ‘ignore’}, 默認為 ‘raise’
配置錯誤處理。

‘ignore’：如果meta中列出的鍵並非總是存在，將忽略KeyError。

‘raise’：如果meta中列出的鍵並非總是存在，將引發KeyError。

sep：str, 默認為 ‘.’
嵌套記錄將生成用sep分隔的名稱。例如，對於sep =’。’，{‘foo’：{‘bar’：0}}-> foo.bar。

max_level：int, 默認為 None
要進行標準化的最大級別數(詞典的深度)。如果為None，則將所有級別標準化。

In [ ]:
# Load json_normalize()
from pandas.io.json import json_normalize    

# Isolate the JSON data from the API response
data =  response.json()

# Flatten business data into a data frame, replace separator
cafes = json_normalize(data["businesses"],
             sep= '_')

# View data
print(cafes.head())

### Handle deeply nested data

In [ ]:
# Load other business attributes and set meta prefix
flat_cafes = json_normalize(data["businesses"],
                            sep="_",
                    		record_path="categories",
                    		meta =['name', 
                                  'alias',  
                                  'rating',
                          		  ['coordinates', 'latitude'], 
                          		  ['coordinates', 'longitude']],
                    		meta_prefix = "biz_")                    #


# View the data
print(flat_cafes.head())

## 4.4 Combining multiple datasets

### 4.4.1 append() 竖向🔗tables

#### df1.append(df2, ignore_index = [ True #忽视pd.DataFrame的默认行index, False])

In [ ]:
# Add an offset parameter to get cafes 51-100
params = {"term": "cafe", 
          "location": "NYC",
          "sort_by": "rating", 
          "limit": 50,
          'offset': 50}

result = requests.get(api_url, headers=headers, params=params)
next_50_cafes = json_normalize(result.json()["businesses"])

# Append the results, setting ignore_index to renumber rows
cafes = top_50_cafes.append(next_50_cafes, ignore_index = True)

# Print shape of cafes
print(cafes.shape)

### 4.4.2 merge() 横向🔗tables， 类似SQL 的join

#### df1.merge(df2, left_on = 'df1.fk', right_on = 'df2.pk' )

In [ ]:
# Merge crosswalk into cafes on their zip code fields
cafes_with_pumas = cafes.merge(crosswalk, left_on = 'location_zip_code', right_on='zipcode')

# Merge pop_data into cafes_with_pumas on puma field
cafes_with_pop =  cafes_with_pumas.merge(pop_data, left_on = 'puma', right_on = 'puma')

# View the data
print(cafes_with_pop.head())